# 🟨 SIMPLI colab 

#### This notebook was composed for demonstrating the SIMPLI model from the paper "Self-improving Multiplane-to-layer Images for Novel View Synthesis".
#### Model SIMPLI works with a set of scene views, each view is an image with camera parameters. As result, it generates a multiplane layer image (MLI) representation, that could be easily rendered by a standard graphics pipeline to obtain novel scene views.

#### This colab includes several steps:


1.   Loading user video. 
2.   Processing frames from video with SFM for obtaining views poses.
3.   Selection views that will be used for MLI from processed frames.
4.   Build MLI with SIMPLI model.
5.   Run online MLI viewer.

#### This colab contains parameters that you can adjust, but it works well with default values for videos that have HD resolution and was shot in landscape orientation. 🟡 highlights important information.

![simpli_scheme](https://github.com/SamsungLabs/MLI/blob/main/docs/resources/simpli_pipeline.jpg?raw=true)



In [ ]:
!pip install -q gwpy       

In [ ]:
#@title The SIMPLI pipeline in colab needs access to your google drive for storing intermediate results and load modules from GitHub.

from google.colab import drive
drive.mount('/content/gdrive')

work_dir_path = '/content/gdrive/MyDrive/simpli_test/captures' # @param {type: 'string'}

## Dependencies

In [ ]:
# @title Instal dependencies
%%capture
!apt-get install colmap ffmpeg
!pip install numpy==1.19.3
!pip install p_tqdm 
!pip install ffmpeg_python
!pip3 install torch torchvision torchaudio

!apt-get install ffmpeg
!pip install Ninja
!pip3 install torch torchvision torchaudio
%cd "/content/drive/My Drive/colab"
!git clone https://github.com/NVlabs/nvdiffrast.git
%cd "nvdiffrast"
!pip install .

%cd ".."

!pip install  tqdm \
              h5py \
              scikit-image \
              einops \
              graphviz \
              plyfile \
              munch \
              huepy \
              Cython \
              tensorboardX \
              packaging \
              torchsummary \
              imageio \
              pyyaml \
              opencv-python \
              pyntcloud \
              lpips \
              kornia==0.5.11 \
              ffmpeg_python \
              pandas \
              pillow 



In [ ]:
%%capture

# @title Dependencies imports 
import os
import sys
import shutil
import json
import gc
from pathlib import Path
from timeit import default_timer as timer
import math

import warnings
warnings.filterwarnings('ignore')

import torch
from PIL import Image


!mkdir -p /content/gdrive/MyDrive/libs
!git clone -b colab https://github.com/palsol/SimpleSfm
!rm -rf /content/gdrive/MyDrive/libs/SimpleSfm
!mv SimpleSfm /content/gdrive/MyDrive/libs

sys.path.append('/content/gdrive/MyDrive/libs/SimpleSfm')

from simple_sfm.scene_utils.video_to_scene_processors import OneVideoSceneProcesser
from simple_sfm.scene_utils.matcher import Matcher
from simple_sfm.scene_utils.colmap_scene_converters import colmap_sparse_to_re10k_like_views
from simple_sfm.scene_utils.colmap_bd_utils import ColmapBdManager
from simple_sfm.scene_utils.scene_readers import read_re10k_views

from simple_sfm.utils.video_streamer import VideoStreamer
from simple_sfm.cameras.camera_multiple import CameraMultiple
from simple_sfm.cameras.utils import average_extrinsics


!git clone https://github.com/SamsungLabs/MLI.git
!rm -rf /content/gdrive/MyDrive/libs/MLI
!mv MLI /content/gdrive/MyDrive/libs

sys.path.append('/content/gdrive/MyDrive/libs/MLI')

from lib.trainers.utils import create_trainer_load_weights_from_config
from lib.utils.io import get_config


# copy viewer extension to nbextensions
!rm -rf /usr/local/share/jupyter/nbextensions/google.colab/multi-layer-viewer
!cp -R /content/gdrive/MyDrive/libs/MLI/docs/viewer \
    /usr/local/share/jupyter/nbextensions/google.colab/multi-layer-viewer

!git clone https://github.com/magicleap/SuperGluePretrainedNetwork.git
!rm -rf /content/gdrive/MyDrive/libs/SuperGluePretrainedNetwork
!mv SuperGluePretrainedNetwork /content/gdrive/MyDrive/libs


superglue_weigths_path = '/content/gdrive/MyDrive/libs/SuperGluePretrainedNetwork/models/weights/superglue_outdoor.pth'
superpoint_weigths_path = '/content/gdrive/MyDrive/libs/SuperGluePretrainedNetwork/models/weights/superpoint_v1.pth'

## Load SIMPLI model.
#### 🟡 You can select a SIMPLI model that generates 8 or 4 layers MLI representation.

In [ ]:

num_layers = '8' #@param ['4', '8']


checkpoints_path = f'/content/gdrive/MyDrive/libs/MLI/pretrained/model{num_layers}_layers/'
config_path = f'/content/gdrive/MyDrive/libs/MLI/pretrained/model{num_layers}_layers/tblock{num_layers}.yaml'

config = get_config(config_path)
iteration = 660000

trainer, loaded_iteration = create_trainer_load_weights_from_config(config=config,
                                                                    checkpoints_dir=checkpoints_path,
                                                                    iteration=iteration,
                                                                    device='cuda'
                                                                    )

_ = trainer.eval()


## 1. Load your video.
#### 🟡 For the best result, you need to use a short landscape-oriented video of a static scene. Choose some object and shoot it by moving the camera in front of the object like in the following picture.
![How to shoot](https://github.com/SamsungLabs/MLI/blob/main/docs/resources/how_to_record_video_new.jpeg?raw=true)

#### 🟡 It is better to use a video that has a resolution of ~ 1280x720. If your video is larger, you can set the **scale_factor** parameter in the ***2. Process video with SFM*** section, and it will be resized.
#### 🟡 You can find a sample video [here](https://github.com/SamsungLabs/MLI/blob/main/docs/resources/sample_video.mp4)
#### 🟡 Our method is designed for working with static scenes without hard specular effects. However, it could work with scenes that contain a reasonable amount of them.
#### 🟡 If you want to experiment with other videos without installing dependencies and reloading models you can choose another video and rerun the notebook from this cell. Enjoy!!!


In [ ]:
from google.colab import files

uploaded = files.upload()

## 2. Process video with SFM

#### 🟡 If your video resolution is larger than 1280x720 is recommended to set the **scale_factor**  so that the rescaled video resolution will be approximately 1280x720. Otherwise, the consumption of GPU memory can be greater than available on the collab machine.

#### 🟡 If your camera shoots video with a significant barrel/pincushion distortion, flag **central_crop** would be helpful.

#### 🟡 You can increase the maximum number of frames **max_num_frames** that will be processed by the SFM pipeline and will farther be available to select for using in SIMPLI model. However, the execution time of the SFM pipeline could be increased

In [ ]:
# v c = '/content/gdrive/MyDrive/simpli_test/20220911_161609.mp4'
# temp_video_path = '/content/gdrive/MyDrive/simpli_test/2022-08-23_test.mp4'
temp_video_path = next(iter(uploaded.keys()))
video_full_name = temp_video_path.split('/')[-1]
capture_name = video_full_name.split('.')[0]


capture_work_dir = Path(work_dir_path, capture_name)
capture_work_dir.mkdir(exist_ok=True, parents=True)
shutil.rmtree(capture_work_dir) 

video_path = Path(capture_work_dir, 'raw_video')
video_path.mkdir(exist_ok=True, parents=True)
shutil.copy(temp_video_path, Path(video_path, video_full_name))  
video_path = Path(video_path, video_full_name)


# # @markdown Frames to skip.
# skip = 1  # @param {type: 'integer'}
skip = 1
# @markdown Amount of frames to use from video.
max_num_frames = 17  # @param {type: 'integer'}
# @markdown Do center crop?
center_crop = False  # @param {type: 'boolean'}
# @markdown Images scale
scale_factor = 1  #@param {type:"slider", min:0.1, max:1, step:0.05}


video_to_frames = OneVideoSceneProcesser(
    video_path=str(video_path),
    dataset_output_path=str(capture_work_dir),
    skip=skip,
    center_crop=center_crop,
    scale_factor=scale_factor,
    max_len=max_num_frames,
    img_prefix='jpg',
    filter_with_sharpness=True,
)

video_to_frames.run()
frames_path = Path(capture_work_dir, 'frames')

### 🟡 Open if you want to play with the matching precedure's parameters.

In [ ]:
# @markdown Non Maximum Suppression (NMS) radius
nms_radius = 4  #@param {type:"slider", min:1, max:10, step:1}
# @markdown Detector confidence threshold.
keypoint_threshold = 0.01  #@param {type:"slider", min:0.001, max:0.1, step:0.001}
# @markdown Threshold value for matching.
match_threshold = 0.75  #@param {type:"slider", min:0.05, max:0.95, step:0.05}
# @markdown Num sinkhorn iterations for super glue matching.
sinkhorn_iterations = 20 #@param {type:"slider", min:1, max:50, step:1}
# @markdown Batch size for super glue infer
super_glue_batch = 5 #@param {type:"slider", min:1, max:10, step:1}



matcher = Matcher(
    super_point_extractor_weights_path=superpoint_weigths_path,
    super_glue_weights_path=superglue_weigths_path,
    nms_radius=nms_radius,
    keypoint_threshold=keypoint_threshold,
    matcher_type='super_glue',
    match_threshold=match_threshold,
    sinkhorn_iterations=sinkhorn_iterations,
    super_glue_batch=int(super_glue_batch)
    )

In [ ]:
%%capture
# @title Run sfm over images
vs = VideoStreamer(
    str(frames_path), 
    height=None, 
    width=None, 
    max_len=None, 
    img_glob='*.jpg'
  )
camera_size = vs.get_resolution()

colmap = ColmapBdManager(
    db_dir=str(Path(capture_work_dir, 'colmap')),
    images_folder_path=str(frames_path),
    camera_type='OPENCV',
    camera_params=None,
    camera_size=camera_size
)

processed_frames, match_table, images_names = matcher.match_video_stream(vs, lambda x: True)
torch.cuda.empty_cache()

colmap.replace_images_data(images_names)
colmap.replace_keypoints(images_names, processed_frames)
colmap.replace_and_verificate_matches(match_table, images_names)
num_sparse_points = colmap.run_mapper()


## 3. Selecting source view for building MLI

#### 🟡 Some scenes have objects of interest too near or too far. You can adjust **translation_scale** for the best result on your scene. However, the default value was OK for most scenes that we tried.


In [ ]:
colmap_sparse_to_re10k_like_views(
    scene_colmap_sparse_path=os.path.join(capture_work_dir, 'colmap', 'sparse'),
    views_file_output_path=capture_work_dir,
    scene_meta_file_output_path=capture_work_dir,
)

# # @markdown Source camera resolution rescale:
# source_camera_scale = 1  #@param {type:"slider", min:0.1, max:1, step:0.05}

source_camera_scale = 1

# @markdown Scene scale:
translation_scale = 1.6 #@param {type:"slider", min:1, max:5, step:0.1}

resize_size = [int(camera_size[1] * source_camera_scale),
             int(camera_size[0] * source_camera_scale)] 


# resize_size = [math.floor(resize_size[0] / 16) * 16,
#                math.floor(resize_size[1] / 16) * 16] 


crop_size = [math.floor(resize_size[0] / 16) * 16,
             math.floor(resize_size[1] / 16) * 16] 

intrinsics, extrinsics, images = read_re10k_views(
    views_file_path=os.path.join(capture_work_dir, 'views.txt'),
    scene_meta_path=os.path.join(capture_work_dir, 'scene_meta.yaml'),
    frames_path=os.path.join(capture_work_dir, 'frames'),
    frames_crop_size=crop_size,
    frames_resize_size=resize_size,
    translation_scale=translation_scale
    )



extrinsics = torch.stack(extrinsics, dim=0).cuda()
intrinsics = torch.stack(intrinsics, dim=0).cuda()
images = torch.stack(images, dim=0).cuda()
source_images_sizes = images.shape[-2:]

all_cameras = CameraMultiple(extrinsics=extrinsics, 
                             intrinsics=intrinsics,
                             images_sizes=[list(images.shape[-2:])] * extrinsics.shape[0]
                            )

# @markdown Select number of views for bulding MLI
num_views = 8  #@param {type:"slider", min:1, max:10, step:1}

def find_most_distant_point_a_to_b(set_a, set_b):
  size_a = set_a.shape[0]
  size_b = set_b.shape[0]
  distances = torch.sqrt(torch.sum((set_b.repeat(size_a, 1) - set_a.repeat(1, size_b).reshape(-1, 3))**2, dim=1))
  min_dist, _ = distances.reshape(size_a, -1).min(dim=1)
  return torch.argmax(min_dist)

def find_most_distant_point(point_set):
  size_point_set = point_set.shape[0]
  distances = torch.sqrt(torch.sum((point_set.repeat(size_point_set, 1) - point_set.repeat(1, size_point_set).reshape(-1, 3))**2, dim=1))
  mask = (distances == 0)
  distances = distances + mask * max(distances)
  min_dist, _ = distances.reshape(size_point_set, -1).min(dim=1)
  return torch.argmax(min_dist)

def get_k_most_distant_cams(cameras, k):
  cam_pos = cameras.world_position
  cam_dir = cameras.world_view_direction()

  first_point_id = find_most_distant_point(cam_pos)
  result = cam_pos[[first_point_id]][None]
  result_id = first_point_id[None]

  k = num_views
  for i in range(k - 1):
    next_point_id = find_most_distant_point_a_to_b(cam_pos, cam_pos[result_id])
    result_id = torch.cat([result_id, next_point_id[None]])

  return cameras[result_id], result_id

selected_cameras, select_id = get_k_most_distant_cams(all_cameras, num_views)
selected_images = images[select_id]

# selected_cameras = all_cameras
# selected_images = images

reference_extrinsic = average_extrinsics(selected_cameras.extrinsics)

# # @markdown Reference camera scale
# ref_camera_scale = 2  #@param {type:"slider", min:0.5, max:2, step:0.05}

ref_camera_scale = 1
ref_intrinsic = intrinsics[:1].clone()
reference_images_sizes = [int(source_images_sizes[0] * ref_camera_scale),
                          int(source_images_sizes[1] * ref_camera_scale)]
reference_images_sizes = [reference_images_sizes[0] // 16 * 16, reference_images_sizes[1] // 16 * 16]
ref_intrinsic[:, :2, :2] = ref_intrinsic[:, :2, :2] / ref_camera_scale


# reference_images_sizes = [720, 1280]
# reference_images_sizes = [reference_images_sizes[0] // 16 * 16, reference_images_sizes[1] // 16 * 16]
# ref_intrinsic = torch.tensor([[[reference_images_sizes[0] / reference_images_sizes[1], 0, 0.5],
#                                [0, 1.0, 0.5],
#                                [0, 0, 1.0]]]).cuda()

reference_camera = CameraMultiple(extrinsics=reference_extrinsic[None, None],
                                  intrinsics=ref_intrinsic[None, None],
                                  images_sizes=reference_images_sizes,
                                  )

## Viewer of scene camera poses 
#### 🟡 Open this section to see a plot of scene cameras and which cameras were selected for building MLI (orange points). :
#### 🟡 The green arrow shows a camera view direction, the blue arrow indicates a camera UP direction.
#### 🟡 The green point is a virtual camera in which a frustum MLI representation will be built.

In [ ]:
# @title ___
import plotly.graph_objects as go
import numpy as np

# @markdown Plot scale:
plot_size = 1  #@param {type:"slider", min:0.01, max:1, step:0.01}

def ploty_plot_extrinsics(fig, extrinsics, axis_scale=0.01, line_width=4, marker_size=1, opacity=1):
    translation = extrinsics[..., :3, -1:]
    rotation = extrinsics[..., :3, :3]

    cameras_world_positions = -rotation.transpose(-1, -2) @ translation
    cameras_world_positions = cameras_world_positions[:, :, 0]
    cameras_R = rotation
    
    fig = fig.add_trace(go.Scatter3d(
                    x=cameras_world_positions[:, 0], 
                    y=cameras_world_positions[:, 1], 
                    z=cameras_world_positions[:, 2],
                    mode="markers+text",
                    marker={"size": marker_size, 'color':'black'},
                    text=[str(el) for el in range(cameras_R.shape[0])],
                    textposition="middle left",
                    opacity=opacity
                    ))
    
    for i in range(cameras_R.shape[0]):
        R, T = cameras_R[i], cameras_world_positions[i]

        fig.add_trace(go.Scatter3d(
            x=[T[0], T[0] + R[0, 0] * axis_scale], 
            y=[T[1], T[1] + R[0, 1] * axis_scale], 
            z=[T[2], T[2] + R[0, 2] * axis_scale], 
            line={"color":'red', 'width':line_width},
            mode='lines', 
            showlegend=False,
            opacity=opacity
        ))
        fig.add_trace(go.Scatter3d(
            x=[T[0], T[0] + R[1, 0] * axis_scale], 
            y=[T[1], T[1] + R[1, 1] * axis_scale], 
            z=[T[2], T[2] + R[1, 2] * axis_scale], 
            line={"color":'green', 'width':line_width},
            mode='lines', 
            showlegend=False,
            opacity=opacity
        ))
        fig.add_trace(go.Scatter3d(
            x=[T[0], T[0] + R[2, 0] * axis_scale], 
            y=[T[1], T[1] + R[2, 1] * axis_scale], 
            z=[T[2], T[2] + R[2, 2] * axis_scale], 
            line={"color":'blue', 'width':line_width},
            mode='lines', 
            showlegend=False,
            opacity=opacity
        ))
        
    
    return fig

  

def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-1.5.1.min.js?noext',
            },
          });
        </script>
        '''))

all_cam_pos = all_cameras.world_position.cpu()
cams_positions = go.Scatter3d(
    x=all_cam_pos[:, 0].numpy(), 
    y=all_cam_pos[:, 1].numpy(), 
    z=all_cam_pos[:, 2].numpy(),
    mode='markers', 
    surfacecolor='#ff0000',
    name='not selected'
    )

selected_cam_pos = selected_cameras.world_position.cpu()
cams_distant = go.Scatter3d(
    x=selected_cam_pos[:, 0].numpy(), 
    y=selected_cam_pos[:, 1].numpy(), 
    z=selected_cam_pos[:, 2].numpy(),
    mode='markers', 
    surfacecolor='#0000ff',
    name='selected'
    )

ref_cam_pos = reference_camera.world_position.cpu()
cams_ref = go.Scatter3d(
    x=ref_cam_pos[0, 0, :, 0].numpy(), 
    y=ref_cam_pos[0, 0, :, 1].numpy(), 
    z=ref_cam_pos[0, 0, :, 2].numpy(),
    mode='markers', 
    surfacecolor='#00ffff',
    name='reference camera'
    )

fig = go.Figure(data=[cams_positions, cams_distant, cams_ref])

fig = ploty_plot_extrinsics(fig, all_cameras.extrinsics.cpu(), opacity=0.5, axis_scale=0.06)

fig.update_layout(
    scene = dict(
        xaxis = dict(nticks=4, range=[-plot_size, plot_size],),
        yaxis = dict(nticks=4, range=[-plot_size, plot_size],),
        zaxis = dict(nticks=4, range=[-plot_size, plot_size],),
    ),
)

fig.update_layout(scene_aspectmode='cube')
fig.show()

## 4. Build MLI with the SIMPLI model

In [ ]:
# @title Run SIMPLI 
%%time
torch.cuda.empty_cache()

trainer.eval()

with torch.no_grad():
  result = trainer.gen.manual_forward(source_images=selected_images[None], 
                                      source_cameras=selected_cameras[None, None],
                                      reference_cameras=reference_camera
                                      )



In [ ]:
! rm -rf '/usr/local/share/jupyter/nbextensions/google.colab/multi-layer-viewer/mli_scene'

In [ ]:
# %%capture
#@title Save MLI
def save_mpi(mpi, path, save_as_jpg=False):
    n_planes, channels, height, width = mpi.shape
    mpi = np.concatenate([mpi[:, :-1] * 0.5 + 0.5, mpi[:, -1:]], axis=1)
    mpi = mpi.transpose((0, 2, 3, 1))
    mpi = (255 * np.clip(mpi, 0, 1)).astype(np.uint8)

    os.makedirs(path, exist_ok=True)
    for i, layer in enumerate(mpi):
        if save_as_jpg:
          Image.fromarray(layer[:, :, :3]).save(os.path.join(path, f'layer_{i:02d}.jpg'), optimize=True)
          Image.fromarray(np.repeat(layer[:, :, -1:], 3, axis=2)).save(os.path.join(path, f'layer_alpha_{i:02d}.jpg'), optimize=True)
        else:
            Image.fromarray(layer).save(os.path.join(path, f'layer_{i:02d}.png'), optimize=True)

def save_layered_depth(layered_depth, path):
  num_layers, h, w = layered_depth.shape
  depth_meta_data = {}
  for i in range(num_layers):
      depth = layered_depth[i]
      low, high = np.min(depth), np.max(depth)
      scaled = (depth - low) / (high - low)
      ui = np.clip(scaled * 256.0, 0, 255).astype(np.uint8)
      img = Image.fromarray(ui)

      tag = f'layer_depth_{i:02d}'
      img.save(os.path.join(path, tag + '.jpg'), quality=100)
      depth_meta_data[tag] = [low.item(), high.item()]

  return depth_meta_data

def write_meta(depth_meta_data, resolution, extrinsic, intrinsic, path):
      meta_data = depth_meta_data

      meta_data.update({'frame_size': resolution,
                        'extrinsic_re':extrinsic,
                        'intrinsics_re':intrinsic,
                        })

      with open(os.path.join(path,'meta.json'), 'w') as f:
        json.dump(meta_data, f, ensure_ascii=True)

geom_path = '/usr/local/share/jupyter/nbextensions/google.colab/multi-layer-viewer/mli_scene/'

mpi = result['mpi'][0, 0].cpu()
layered_depth = result['layered_depth'][0].cpu()

save_mpi(mpi, geom_path, save_as_jpg=True)
depth_meta_data = save_layered_depth(layered_depth.numpy(), geom_path)

intr = reference_camera.intrinsics[0].flatten().cpu().numpy()
extr = reference_camera.extrinsics[0].flatten().cpu().numpy()
write_meta(depth_meta_data, 
           [layered_depth.shape[1], layered_depth.shape[2]], 
           list(extr.astype(float)), 
           list(intr.astype(float)), 
           geom_path)

final_mli_path = os.path.join(capture_work_dir, f'{capture_name}_mli.zip')
final_geom_path = os.path.join(geom_path, '*')
! zip -9jpr {final_mli_path} {final_geom_path}


In [ ]:
print(f'You can find saved MLI here :{final_mli_path}')

## MLI viewer

In [ ]:
#@title ___
import IPython
display(IPython.core.display.HTML(f'''
      <head>
    <title>Multi-Layer Viewer</title>
    <link rel="stylesheet" href="/nbextensions/google.colab/multi-layer-viewer/viewer.css">
  </head>

   <head>
    <title>Multi-Layer Viewer</title>
    <link rel="stylesheet" href="/nbextensions/google.colab/multi-layer-viewer/viewer/viewer.css">

    <script src="/nbextensions/google.colab/multi-layer-viewer/threejs_bin.js"></script>
    <script src="/nbextensions/google.colab/multi-layer-viewer/papaparse.min.js"></script>
    <script src="https://cdn.jsdelivr.net/npm/@tensorflow/tfjs/dist/tf.min.js"> </script>
    <script src="https://mrdoob.github.io/stats.js/build/stats.min.js"></script>
    <script src="/nbextensions/google.colab/multi-layer-viewer/numjs.min.js"></script>
    <script src="/nbextensions/google.colab/multi-layer-viewer/utils.js"></script>
    <script src="/nbextensions/google.colab/multi-layer-viewer/camera_viewer.js"></script>
    <script src="/nbextensions/google.colab/multi-layer-viewer/materials.js"></script>
    <script src="/nbextensions/google.colab/multi-layer-viewer/viewer.js"></script>

  </head>

   <body>
    <div class="container">

      <div class="title" id="title">
        <h1>Multi-Layer Viewer</h1>
      </div>

<div class="main_controls">
      <div class="controls" style="padding-top: 20px;">
        <div><p style="font-size: 13pt; line-height: 0.05">Movement mode:</p></div>
        <div class="column_">
             <button id="hoverButton">Hover</button>
             <div style="width: 10px; height: 3px"></div>
             <button id="dragButton">Drag</button>
             <div style="width: 10px; height: 3px"></div>
             <button id="swayButton">Sway</button>
             <div style="width: 10px; height: 3px"></div>
             <button id="wonderButton">Wonder</button>
         </div>
      </div>

      <div class="controls" style="padding-top: 30px;">
        <div><p style="font-size: 13pt; line-height: 0.05">View mode:</p></div>
        <div class="column_">
            <button id="magmaViewButton">Depth (Magma)</button>
            <div style="width: 10px; height: 3px"></div>
            <button id="rainbowViewButton">Depth (Rainbow)</button>
            <div style="width: 10px; height: 3px"></div>
            <button id="normalViewButton">Normal</button>
        </div>
      </div>

</div>


        <div class="main-viewer" style="padding-left: 23%; margin-top: -29%;">
           <div class="viewer-container">
        <div id="scene-viewer" class="scene-lf">
         <canvas id="viewer-canvas" class="viewer-canvas" width="994" height="596">
            <param id="base-path" value="../scenes/3/">
          <div id="display"></div>
                 <script>
                document.getElementById("display").innerHTML = startDisplay("/nbextensions/google.colab/multi-layer-viewer/mli_scene", num_layers={num_layers});
              </script>

         </canvas>
        </div>
      </div>
      </div>

    </div>

  </body>
      '''))